# Quadrotor Dynamics

This notebook explores the `dynamics.py` module, which implements:
- Continuous-time rigid body dynamics for a quadrotor
- Quaternion-based attitude representation
- Euler and RK4 integration methods
- Control saturation

## State Representation
The quadrotor state consists of:
- **Position** `p`: 3D position in world frame [m]
- **Velocity** `v`: 3D velocity in world frame [m/s]  
- **Attitude** `q`: Quaternion [w, x, y, z] (scalar-first)
- **Angular velocity** `w_body`: Body-frame angular velocity [rad/s]

## Control Inputs
- **Thrust** `T`: Total thrust magnitude [N]
- **Moments** `τ`: Body-frame torques [N·m]

In [ ]:
# Setup: Add the src directory to Python path
import sys
sys.path.insert(0, '../src')

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Import quadrotor modules
from quad.dynamics import omega_matrix, state_derivative, step_euler, step_rk4, saturate_control
from quad.types import State, Control
from quad.params import Params
from quad.math3d import quat_to_euler, quat_normalize

# Set up plotting style
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['figure.dpi'] = 100

print("Modules loaded successfully!")

## 1. The Omega Matrix

The quaternion kinematics are given by:
$$\dot{q} = \frac{1}{2} \Omega(\omega) q$$

where $\Omega(\omega)$ is a 4×4 skew-symmetric matrix constructed from the angular velocity vector $\omega = [\omega_x, \omega_y, \omega_z]^T$.

This formulation propagates the quaternion attitude based on the body angular velocity.

In [ ]:
# Explore the Omega matrix structure
w = np.array([1.0, 2.0, 3.0])  # Example angular velocity [rad/s]
Omega = omega_matrix(w)

print("Angular velocity ω =", w)
print("\nOmega(ω) matrix:")
print(Omega)

# Verify skew-symmetry: Omega = -Omega^T
print("\nIs Omega skew-symmetric? (Omega + Omega.T should be zero)")
print(Omega + Omega.T)

## 2. State Derivatives

The complete dynamics equations are:

**Position dynamics:**
$$\dot{p} = v$$

**Velocity dynamics:**
$$\dot{v} = \begin{bmatrix} 0 \\ 0 \\ -g \end{bmatrix} + \frac{1}{m} R(q) \begin{bmatrix} 0 \\ 0 \\ T \end{bmatrix} - k_d v$$

**Attitude kinematics:**
$$\dot{q} = \frac{1}{2} \Omega(\omega_{body}) q$$

**Angular velocity dynamics (Euler's equation):**
$$\dot{\omega} = J^{-1} \left( \tau - \omega \times (J\omega) \right)$$

In [ ]:
# Create default parameters and initial state
params = Params()
print("Quadrotor Parameters:")
print(f"  Mass: {params.m} kg")
print(f"  Gravity: {params.g} m/s²")
print(f"  Hover thrust: {params.hover_thrust:.2f} N")
print(f"  Inertia (diagonal): {np.diag(params.J)}")

# Initial state: hovering at 1m altitude
state = State.zeros()
state.p = np.array([0.0, 0.0, 1.0])  # 1m altitude

print(f"\nInitial State:")
print(f"  Position: {state.p}")
print(f"  Velocity: {state.v}")
print(f"  Quaternion: {state.q} (identity = level attitude)")
print(f"  Angular velocity: {state.w_body}")

In [ ]:
# Test 1: Hover equilibrium
# With hover thrust, acceleration should be zero
control_hover = Control(thrust_N=params.hover_thrust, moments_Nm=np.zeros(3))
p_dot, v_dot, q_dot, w_dot = state_derivative(state, control_hover, params)

print("=== Hover Equilibrium Test ===")
print(f"Thrust: {control_hover.thrust_N:.2f} N (hover thrust)")
print(f"\nState derivatives:")
print(f"  ṗ (velocity): {p_dot}")
print(f"  v̇ (acceleration): {v_dot}")
print(f"  q̇ (quaternion rate): {q_dot}")
print(f"  ω̇ (angular acceleration): {w_dot}")
print(f"\nAs expected, v̇ ≈ 0 (hovering) and ω̇ = 0 (no rotation)")

In [ ]:
# Test 2: Free fall (zero thrust)
control_zero = Control(thrust_N=0.0, moments_Nm=np.zeros(3))
p_dot, v_dot, q_dot, w_dot = state_derivative(state, control_zero, params)

print("=== Free Fall Test ===")
print(f"Thrust: 0.0 N")
print(f"\nState derivatives:")
print(f"  v̇ (acceleration): {v_dot}")
print(f"  Expected: [0, 0, {-params.g:.4f}] (gravity)")
print(f"\nThe quadrotor accelerates downward at g = {params.g} m/s²")

## 3. Integration Methods: Euler vs RK4

The module provides two integration methods:
- **Forward Euler**: Simple but less accurate, $x_{k+1} = x_k + \Delta t \cdot f(x_k)$
- **RK4**: 4th-order Runge-Kutta, much more accurate for the same timestep

Let's compare them by simulating upward acceleration.

In [ ]:
# Compare Euler vs RK4 for a simple scenario
# Simulate upward thrust causing the quad to rise

dt = 0.01  # 10ms timestep
T_sim = 3.0  # 3 seconds
n_steps = int(T_sim / dt)

# Initial state: at rest on the ground
state_euler = State.zeros()
state_rk4 = State.zeros()

# Slight thrust above hover to accelerate upward
control = Control(thrust_N=params.hover_thrust * 1.2, moments_Nm=np.zeros(3))

# Storage for plotting
t_hist = np.zeros(n_steps)
z_euler = np.zeros(n_steps)
z_rk4 = np.zeros(n_steps)
vz_euler = np.zeros(n_steps)
vz_rk4 = np.zeros(n_steps)

# Simulate
for i in range(n_steps):
    t_hist[i] = i * dt
    z_euler[i] = state_euler.p[2]
    z_rk4[i] = state_rk4.p[2]
    vz_euler[i] = state_euler.v[2]
    vz_rk4[i] = state_rk4.v[2]
    
    state_euler = step_euler(state_euler, control, params, dt)
    state_rk4 = step_rk4(state_rk4, control, params, dt)

# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(t_hist, z_euler, 'b-', label='Euler', alpha=0.7)
axes[0].plot(t_hist, z_rk4, 'r--', label='RK4', linewidth=2)
axes[0].set_xlabel('Time [s]')
axes[0].set_ylabel('Altitude [m]')
axes[0].set_title('Position: Euler vs RK4')
axes[0].legend()
axes[0].grid(True)

axes[1].plot(t_hist, vz_euler, 'b-', label='Euler', alpha=0.7)
axes[1].plot(t_hist, vz_rk4, 'r--', label='RK4', linewidth=2)
axes[1].set_xlabel('Time [s]')
axes[1].set_ylabel('Vertical Velocity [m/s]')
axes[1].set_title('Velocity: Euler vs RK4')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

print(f"Final altitude - Euler: {state_euler.p[2]:.4f} m, RK4: {state_rk4.p[2]:.4f} m")
print(f"Difference: {abs(state_euler.p[2] - state_rk4.p[2]):.6f} m")

## 4. Rotational Dynamics

Let's explore the rotational dynamics by applying torques and observing the attitude evolution.

In [ ]:
# Simulate rotation with applied torque
dt = 0.002  # Smaller timestep for rotational dynamics
T_sim = 2.0
n_steps = int(T_sim / dt)

# Start from hover
state = State.zeros()
state.p = np.array([0.0, 0.0, 1.0])

# Apply roll torque for first half, then stop
tau_roll = np.array([0.02, 0.0, 0.0])  # Roll torque

# Storage
t_hist = np.zeros(n_steps)
euler_angles = np.zeros((n_steps, 3))
omega_hist = np.zeros((n_steps, 3))
q_norm_hist = np.zeros(n_steps)

for i in range(n_steps):
    t = i * dt
    t_hist[i] = t
    
    # Record state
    euler_angles[i] = np.degrees(quat_to_euler(state.q))
    omega_hist[i] = state.w_body
    q_norm_hist[i] = np.linalg.norm(state.q)
    
    # Apply torque for first second only
    if t < 1.0:
        control = Control(thrust_N=params.hover_thrust, moments_Nm=tau_roll)
    else:
        control = Control(thrust_N=params.hover_thrust, moments_Nm=np.zeros(3))
    
    state = step_rk4(state, control, params, dt)

# Plot results
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Euler angles
axes[0, 0].plot(t_hist, euler_angles[:, 0], 'r-', label='Roll')
axes[0, 0].plot(t_hist, euler_angles[:, 1], 'g-', label='Pitch')
axes[0, 0].plot(t_hist, euler_angles[:, 2], 'b-', label='Yaw')
axes[0, 0].axvline(x=1.0, color='k', linestyle='--', alpha=0.5, label='Torque off')
axes[0, 0].set_xlabel('Time [s]')
axes[0, 0].set_ylabel('Angle [deg]')
axes[0, 0].set_title('Euler Angles')
axes[0, 0].legend()
axes[0, 0].grid(True)

# Angular velocities
axes[0, 1].plot(t_hist, omega_hist[:, 0], 'r-', label='ωx (roll)')
axes[0, 1].plot(t_hist, omega_hist[:, 1], 'g-', label='ωy (pitch)')
axes[0, 1].plot(t_hist, omega_hist[:, 2], 'b-', label='ωz (yaw)')
axes[0, 1].axvline(x=1.0, color='k', linestyle='--', alpha=0.5, label='Torque off')
axes[0, 1].set_xlabel('Time [s]')
axes[0, 1].set_ylabel('Angular Velocity [rad/s]')
axes[0, 1].set_title('Body Angular Velocities')
axes[0, 1].legend()
axes[0, 1].grid(True)

# Quaternion norm (should stay at 1)
axes[1, 0].plot(t_hist, q_norm_hist - 1.0, 'b-')
axes[1, 0].set_xlabel('Time [s]')
axes[1, 0].set_ylabel('||q|| - 1')
axes[1, 0].set_title('Quaternion Norm Deviation')
axes[1, 0].grid(True)
axes[1, 0].ticklabel_format(style='scientific', axis='y', scilimits=(0,0))

# Note about conservation
axes[1, 1].text(0.5, 0.5, 'After torque stops (t > 1s):\nAngular velocity stays constant\n(no external torques)', 
                transform=axes[1, 1].transAxes, ha='center', va='center', fontsize=12)
axes[1, 1].set_title('Conservation of Angular Momentum')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print(f"Final roll rate: {omega_hist[-1, 0]:.4f} rad/s")
print(f"Quaternion norm deviation: {abs(q_norm_hist[-1] - 1.0):.2e}")

## 5. Control Saturation

Real actuators have limits. The `saturate_control` function clips thrust and moments to physical limits.

In [ ]:
# Demonstrate control saturation
print("Actuator Limits:")
print(f"  Thrust: [{params.T_min}, {params.T_max}] N")
print(f"  Moments: ±{params.tau_max} N·m")

# Create an aggressive control command
control_aggressive = Control(
    thrust_N=50.0,  # Way above max
    moments_Nm=np.array([0.5, -0.3, 0.2])  # Above limits
)

# Saturate it
control_saturated = saturate_control(control_aggressive, params)

print(f"\nOriginal control:")
print(f"  Thrust: {control_aggressive.thrust_N} N")
print(f"  Moments: {control_aggressive.moments_Nm} N·m")

print(f"\nSaturated control:")
print(f"  Thrust: {control_saturated.thrust_N} N")
print(f"  Moments: {control_saturated.moments_Nm} N·m")

## 6. Full Simulation: Tilted Quad with Thrust

Let's simulate a more complex scenario: a tilted quadrotor with thrust, showing coupled translational and rotational dynamics.

In [ ]:
# Simulate a tilted quadrotor
dt = 0.005
T_sim = 3.0
n_steps = int(T_sim / dt)

# Start tilted 30 degrees in pitch
pitch_angle = np.radians(30)
q_pitched = np.array([np.cos(pitch_angle/2), 0, np.sin(pitch_angle/2), 0])

state = State(
    p=np.array([0.0, 0.0, 5.0]),  # Start at 5m altitude
    v=np.zeros(3),
    q=q_pitched,
    w_body=np.zeros(3)
)

# Constant thrust (hover thrust - not enough when tilted!)
control = Control(thrust_N=params.hover_thrust, moments_Nm=np.zeros(3))

# Storage
t_hist = np.zeros(n_steps)
pos_hist = np.zeros((n_steps, 3))
vel_hist = np.zeros((n_steps, 3))
euler_hist = np.zeros((n_steps, 3))

for i in range(n_steps):
    t_hist[i] = i * dt
    pos_hist[i] = state.p
    vel_hist[i] = state.v
    euler_hist[i] = np.degrees(quat_to_euler(state.q))
    
    state = step_rk4(state, control, params, dt)

# Plot
fig = plt.figure(figsize=(14, 10))

# 3D trajectory
ax3d = fig.add_subplot(2, 2, 1, projection='3d')
ax3d.plot(pos_hist[:, 0], pos_hist[:, 1], pos_hist[:, 2], 'b-', linewidth=2)
ax3d.scatter(pos_hist[0, 0], pos_hist[0, 1], pos_hist[0, 2], c='g', s=100, label='Start')
ax3d.scatter(pos_hist[-1, 0], pos_hist[-1, 1], pos_hist[-1, 2], c='r', s=100, label='End')
ax3d.set_xlabel('X [m]')
ax3d.set_ylabel('Y [m]')
ax3d.set_zlabel('Z [m]')
ax3d.set_title('3D Trajectory (Tilted Quad)')
ax3d.legend()

# Position over time
ax1 = fig.add_subplot(2, 2, 2)
ax1.plot(t_hist, pos_hist[:, 0], 'r-', label='X')
ax1.plot(t_hist, pos_hist[:, 1], 'g-', label='Y')
ax1.plot(t_hist, pos_hist[:, 2], 'b-', label='Z')
ax1.set_xlabel('Time [s]')
ax1.set_ylabel('Position [m]')
ax1.set_title('Position vs Time')
ax1.legend()
ax1.grid(True)

# Velocity over time
ax2 = fig.add_subplot(2, 2, 3)
ax2.plot(t_hist, vel_hist[:, 0], 'r-', label='Vx')
ax2.plot(t_hist, vel_hist[:, 1], 'g-', label='Vy')
ax2.plot(t_hist, vel_hist[:, 2], 'b-', label='Vz')
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Velocity [m/s]')
ax2.set_title('Velocity vs Time')
ax2.legend()
ax2.grid(True)

# Euler angles (should stay constant - no torques)
ax3 = fig.add_subplot(2, 2, 4)
ax3.plot(t_hist, euler_hist[:, 0], 'r-', label='Roll')
ax3.plot(t_hist, euler_hist[:, 1], 'g-', label='Pitch')
ax3.plot(t_hist, euler_hist[:, 2], 'b-', label='Yaw')
ax3.set_xlabel('Time [s]')
ax3.set_ylabel('Angle [deg]')
ax3.set_title('Euler Angles (constant - no torques)')
ax3.legend()
ax3.grid(True)

plt.tight_layout()
plt.show()

print(f"\nWith 30° pitch and hover thrust:")
print(f"  Thrust vector is tilted, so quad drifts forward (X) and falls (Z)")
print(f"  Final position: [{pos_hist[-1, 0]:.2f}, {pos_hist[-1, 1]:.2f}, {pos_hist[-1, 2]:.2f}] m")

## Summary

The `dynamics.py` module provides:

1. **`omega_matrix(w)`**: Constructs the 4×4 quaternion derivative matrix
2. **`state_derivative(state, control, params)`**: Computes all state derivatives
3. **`step_euler(state, control, params, dt)`**: Forward Euler integration
4. **`step_rk4(state, control, params, dt)`**: 4th-order Runge-Kutta integration (recommended)
5. **`saturate_control(control, params)`**: Applies actuator limits

### Key Physics
- Thrust acts along the body z-axis
- Gravity acts downward in the world frame
- Quaternion kinematics preserve unit norm through explicit normalization
- Euler's equation governs rotational dynamics with gyroscopic coupling

### Next Steps
- Explore the `controller_se3.py` module for attitude and position control
- See `trajectory.py` for generating reference trajectories
- Run `main.py` for a full closed-loop simulation